In [1]:
!pip install spacy==2.1.4
!pip install regex
!pip install scikit-learn

  Using cached spacy-2.1.4.tar.gz (29.8 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'error'


  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [316 lines of output]
        Using cached setuptools-68.2.2-py3-none-any.whl.metadata (6.3 kB)
        Using cached wheel-0.41.2-py3-none-any.whl.metadata (2.2 kB)
        Using cached Cython-3.0.4-cp38-cp38-win_amd64.whl.metadata (3.2 kB)
        Using cached cymem-2.0.8-cp38-cp38-win_amd64.whl.metadata (8.6 kB)
        Using cached preshed-2.0.1.tar.gz (113 kB)
        Preparing metadata (setup.py): started
        Preparing metadata (setup.py): finished with status 'done'
        Using cached murmurhash-1.0.10-cp38-cp38-win_amd64.whl.metadata (2.0 kB)
        Using cached thinc-7.0.0.dev6.tar.gz (1.9 MB)
        Preparing metadata (setup.py): started
        Preparing metadata (setup.py): finished with status 'done'
        Using cached blis-0.2.4.tar.gz (1.5 MB)
        Preparing metadata (setup.py): started
        Preparing metadata (setup.py

# Training Dataturks Model with Spacy

In [3]:
############################################  NOTE  ########################################################
#
#           Creates NER training data in Spacy format from JSON downloaded from Dataturks.
#
#           Outputs the Spacy training data which can be used for Spacy training.
#
############################################################################################################
import json
import random
import logging
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from spacy.gold import GoldParse
from spacy.scorer import Scorer
from sklearn.metrics import accuracy_score
import regex as re

def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
    try:
        training_data = []
        lines=[]
        with open(dataturks_JSON_FilePath, 'r', encoding = 'utf-8') as f:
            lines = f.readlines()

        for line in lines:
            data = json.loads(line)
            text = data['content']
            entities = []
            for annotation in data['annotation']:
                #only a single point in text annotation.
                point = annotation['points'][0]
                labels = annotation['label']
                # handle both list of labels or a single label.
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    #dataturks indices are both inclusive [start, end] but spacy is not [start, end)
                    entities.append((point['start'], point['end'] + 1 ,label))


            training_data.append((text, {"entities" : entities}))

        return training_data
    except Exception as e:
        logging.exception("Unable to process " + dataturks_JSON_FilePath + "\n" + "error = " + str(e))
        return None

import spacy

def clean_entities(training_data):
  clean_data = []
  for text, annotation in training_data:
        
    entities = annotation.get('entities')
    entities_copy = entities.copy()
        
    # append entity only if it is longer than its overlapping entity
    i = 0
    for entity in entities_copy:
      j = 0
      for overlapping_entity in entities_copy:
        # Skip self
        if i != j:
          e_start, e_end, oe_start, oe_end = entity[0], entity[1], overlapping_entity[0], overlapping_entity[1]
          # Delete any entity that overlaps, keep if longer
          if ((e_start >= oe_start and e_start <= oe_end) \
          or (e_end <= oe_end and e_end >= oe_start)) \
          and ((e_end - e_start) <= (oe_end - oe_start)):
            entities.remove(entity)
        j += 1
      i += 1
    clean_data.append((text, {'entities': entities}))
                
  return clean_data

def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])

    return cleaned_data
################### Train Spacy NER.###########

def train_spacy():

    TRAIN_DATA = convert_dataturks_to_spacy("data/traindata.json")
    TRAIN_DATA = clean_entities(TRAIN_DATA)
    TRAIN_DATA = trim_entity_spans(TRAIN_DATA)
    nlp = spacy.blank('en')  # create blank Language class
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
       

    # add labels
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(10):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)
    #test the model and evaluate it
    examples = convert_dataturks_to_spacy("data/testdata.json")
    examples = clean_entities(examples)
    examples = trim_entity_spans(examples)
    tp=0
    tr=0
    tf=0

    ta=0
    c=0        
    for text,annot in examples:

        f=open("resume"+str(c)+".txt","w",encoding="utf-8")
        doc_to_test=nlp(text)
        d={}
        for ent in doc_to_test.ents:
            d[ent.label_]=[]
        for ent in doc_to_test.ents:
            d[ent.label_].append(ent.text)

        for i in set(d.keys()):

            f.write("\n\n")
            f.write(i +":"+"\n")
            for j in set(d[i]):
                f.write(j.replace('\n','')+"\n")
        d={}
        for ent in doc_to_test.ents:
            d[ent.label_]=[0,0,0,0,0,0]
        for ent in doc_to_test.ents:
            doc_gold_text= nlp.make_doc(text)
            gold = GoldParse(doc_gold_text, entities=annot.get("entities"))
            y_true = [ent.label_ if ent.label_ in x else 'Not '+ent.label_ for x in gold.ner]
            y_pred = [x.ent_type_ if x.ent_type_ ==ent.label_ else 'Not '+ent.label_ for x in doc_to_test]  
            if(d[ent.label_][0]==0):
                #f.write("For Entity "+ent.label_+"\n")   
                #f.write(classification_report(y_true, y_pred)+"\n")
                (p,r,f,s)= precision_recall_fscore_support(y_true,y_pred,average='weighted')
                a=accuracy_score(y_true,y_pred)
                d[ent.label_][0]=1
                d[ent.label_][1]+=p
                d[ent.label_][2]+=r
                d[ent.label_][3]+=f
                d[ent.label_][4]+=a
                d[ent.label_][5]+=1
        c+=1
    for i in d:
        print("\n For Entity "+i+"\n")
        print("Accuracy : "+str((d[i][4]/d[i][5])*100)+"%")
        print("Precision : "+str(d[i][1]/d[i][5]))
        print("Recall : "+str(d[i][2]/d[i][5]))
        print("F-score : "+str(d[i][3]/d[i][5]))
    nlp.to_disk("models/NER")

train_spacy()

C:\Users\darry\AppData\Roaming\Python\Python38\site-packages\spacy\language.py:632: UserWarning: [W033] Training a new parser or NER using a model with no lexeme normalization table. This may degrade the performance of the model to some degree. If this is intentional or the language you're using doesn't have a normalization table, please ignore this warning. If this is surprising, make sure you have the spacy-lookups-data package installed. The languages with lexeme normalization tables are currently: da, de, el, en, id, lb, pt, ru, sr, ta, th.
  proc.begin_training(


Statring iteration 0


C:\Users\darry\AppData\Roaming\Python\Python38\site-packages\spacy\language.py:479: UserWarning: [W030] Some entities could not be aligned in the text "Navas Koya
Test Engineer

Mangalore, Karnataka - E..." with entities "[[2110, 2404, 'Skills'], [2055, 2064, 'Location'],...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\darry\AppData\Roaming\Python\Python38\site-packages\spacy\language.py:479: UserWarning: [W030] Some entities could not be aligned in the text "Ravi Shivgond
Bidar, Karnataka - Email me on Indee..." with entities "[[1341, 1384, 'Email Address'], [1131, 1136, 'Loca...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\darry\AppData\Roaming\Python\Python38\site-packages\spacy\langu

{'ner': 28639.70567137951}
Statring iteration 1
{'ner': 22160.66072130258}
Statring iteration 2
{'ner': 15826.10133834582}
Statring iteration 3
{'ner': 12635.355202430075}
Statring iteration 4
{'ner': 12337.58576947356}
Statring iteration 5
{'ner': 12273.12476541707}
Statring iteration 6
{'ner': 11257.516649152403}
Statring iteration 7
{'ner': 10254.06506218983}
Statring iteration 8
{'ner': 9651.751251763857}
Statring iteration 9
{'ner': 9536.008587654447}


c:\Users\darry\anaconda3\envs\FYP\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\darry\AppData\Local\Temp\ipykernel_22524\1251979625.py:172: UserWarning: [W030] Some entities could not be aligned in the text "Afreen Jamadar
Active member of IIIT Committee in ..." with entities "[[1155, 1199, 'Email Address'], [743, 1141, 'Skill...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc_gold_text, entities=annot.get("entities"))
C:\Users\darry\AppData\Local\Temp\ipykernel_22524\1251979625.py:172: UserWarning: [W030] Some entities could not be aligned in the text "Afreen Jamadar
Active member of IIIT Committee in ..." with enti


 For Entity Name

Accuracy : 99.83753046303818%
Precision : 0.9983779485610078
Recall : 0.9983753046303818
F-score : 0.9981051835896948

 For Entity Location

Accuracy : 99.51259138911455%
Precision : 0.9935144381746567
Recall : 0.9951259138911455
F-score : 0.9939113647242488

 For Entity Email Address

Accuracy : 99.75629569455727%
Precision : 1.0
Recall : 0.9975629569455727
F-score : 0.9987799918666124

 For Entity Designation

Accuracy : 100.0%
Precision : 1.0
Recall : 1.0
F-score : 1.0

 For Entity Companies worked at

Accuracy : 99.43135662063362%
Precision : 0.9937142688239823
Recall : 0.9943135662063363
F-score : 0.99386812868832

 For Entity Graduation Year

Accuracy : 99.83753046303818%
Precision : 0.9983753046303818
Recall : 0.9983753046303818
F-score : 0.9983753046303818

 For Entity College Name

Accuracy : 100.0%
Precision : 1.0
Recall : 1.0
F-score : 1.0

 For Entity Skills

Accuracy : 100.0%
Precision : 1.0
Recall : 1.0
F-score : 1.0


# Loading Prediction Model

In [6]:
# imports and load spacy english language package
import spacy
from spacy import displacy
from spacy import tokenizer
nlp = spacy.load('models/NER')
nlp.add_pipe(nlp.create_pipe('sentencizer'))
#Load the text and process it
# I copied the text from python wiki
text = '''
8003, Zürich, Switzerland 
+41772789314
charlotte.clerx99@gmail.com
Swiss B permit holder

04.11.1999






•  Strategic forecasting
•  Project management
•  Storytelling and sales pitches

Charlotte Clerx, M.Sc.


Relevant Skills

.

•  Customer relationship management
•
Facilitation of customer UX journeys
•  Market and product segmentation

Professional Experience

•














Mar 2023 - Current                              Internship Strategic Office & Global Ventures - Zurich Insurance            Zurich, Switzerland 


Sep 2022 - Mar 2023                        Internship Strategy & Innovation Consultant - Capgemini                        Zurich, Switzerland   

•  Shaped the 2025 Strategy for the Group Audit of 200 people to increase digitalization and

simplicity by 30% and built a multi-channel communication plan.

•  Redesigned GA’s internal information sharing platforms to reduce complexity and facilitate the

•  Compiled technological and sustainability trends in 200+ companies to keep abreast with

onboarding experience.

new products, and working processes.

•  Created the storyline of the sales pitch for a digital employee benefit platform providing
insurance, wellbeing, discounting 500k gig workers in 33 countries, expected to break-
even by 2025.

•  Marketed 5+ seminars and organized the general assemblies for InsureTech

•  Drove a project to create an incubator program with ETH students to jointly create and

grow social impact ventures.

•  Delivered 20+ RFP’s in digital transformation with consistent record of meeting timeline

scope and budget requirements for 3 - 15 billion revenue clients.

Facilitated the customer journey with a leading AGtech to identify a new Martech strategy.

Educational Background

Sep 2021 - Jan 2023                        M.Sc : International Management - Universidade NOVA SBE              Lisbon, Portugal

Sep 2021 - Jan 2023                       M.Sc : Strategy & Innovation - Maastricht University                         Maastricht, Holland      

•  Coursework in risk- and portfolio management, M&A, strategic business expansion.
•
•  GPA: 8.55/10.

Thesis: The impact of leadership on talent retention strategies.

•  Coursework in statistical modelling, thought leadership.
•  GPA: 7.5/10.

Sep 2018 - Jun 2021                       B.Sc : International Business - Maastricht University                         Maastricht, Holland     

•  Coursework in capital and VC funding, due diligence.
•  Erasmus semester at SGH
•  GPA: 8.2/10.



                                 Warsaw, Poland

Languages

English: full proficiency, German, French, and Spanish: Upper Intermediate, Dutch: Native (EF language exchange in Barcelona,
Madrid, Cambridge, San Diego, Nice)

Core Skills
Microsoft Office Suite, people and communication skills, leadership and can-do attitude, time management

Outdoors sports and fondue, TED talks, organizing own flee market, maintaining several corporate social media accounts

Interests & Other achievements

Volunteering activities



Member of PREMIUM excellence program, the Social Investment Club, and the Beta Gamma Sigma society for honoured students'''


# text2 = # copy the paragraphs from  https://www.python.org/doc/essays/ 
doc = nlp(text)
#doc2 = nlp(text2)
sentences = list(doc.sents)
# print(sentences)
# tokenization
# for token in doc:
#     print(token.text)
# print entities
ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]
print(ents)
# now we use displaycy function on doc2
displacy.render(doc, style='ent', jupyter=True)

[('8003', 1, 5, 'Location'), ('RFP’s in digital transformation with consistent record of meeting timeline', 1504, 1578, 'College Name'), ('English: full proficiency, German, French, and Spanish: Upper Intermediate, Dutch: Native (EF language exchange in Barcelona,\nMadrid, Cambridge, San Diego, Nice)\n\nCore Skills\nMicrosoft Office Suite, people and communication skills, leadership and can-do attitude, time management\n\nOutdoors sports and fondue, TED talks, organizing own flee market, maintaining several corporate social media accounts\n\nInterests & Other achievements\n\nVolunteering activities\n\n\n\nMember of PREMIUM excellence program, the Social Investment Club, and the Beta Gamma Sigma society for honoured students', 2594, 3174, 'Skills')]


# SkillNER

In [7]:
pip install skillNer

Note: you may need to restart the kernel to use updated packages.


In [8]:
# imports
import spacy
from spacy.matcher import PhraseMatcher

# load default skills data base
from skillNer.general_params import SKILL_DB
# import skill extractor
from skillNer.skill_extractor_class import SkillExtractor

# init params of skill extractor
nlp = spacy.load("en_core_web_lg")
# init skill extractor
skill_extractor = SkillExtractor(nlp, SKILL_DB, PhraseMatcher)

# extract skills from job_description
job_description = '''
8003, Zürich, Switzerland 
+41772789314
charlotte.clerx99@gmail.com
Swiss B permit holder

04.11.1999






•  Strategic forecasting
•  Project management
•  Storytelling and sales pitches

Charlotte Clerx, M.Sc.


Relevant Skills

.

•  Customer relationship management
•
Facilitation of customer UX journeys
•  Market and product segmentation

Professional Experience

•














Mar 2023 - Current                              Internship Strategic Office & Global Ventures - Zurich Insurance            Zurich, Switzerland 


Sep 2022 - Mar 2023                        Internship Strategy & Innovation Consultant - Capgemini                        Zurich, Switzerland   

•  Shaped the 2025 Strategy for the Group Audit of 200 people to increase digitalization and

simplicity by 30% and built a multi-channel communication plan.

•  Redesigned GA’s internal information sharing platforms to reduce complexity and facilitate the

•  Compiled technological and sustainability trends in 200+ companies to keep abreast with

onboarding experience.

new products, and working processes.

•  Created the storyline of the sales pitch for a digital employee benefit platform providing
insurance, wellbeing, discounting 500k gig workers in 33 countries, expected to break-
even by 2025.

•  Marketed 5+ seminars and organized the general assemblies for InsureTech

•  Drove a project to create an incubator program with ETH students to jointly create and

grow social impact ventures.

•  Delivered 20+ RFP’s in digital transformation with consistent record of meeting timeline

scope and budget requirements for 3 - 15 billion revenue clients.

Facilitated the customer journey with a leading AGtech to identify a new Martech strategy.

Educational Background

Sep 2021 - Jan 2023                        M.Sc : International Management - Universidade NOVA SBE              Lisbon, Portugal

Sep 2021 - Jan 2023                       M.Sc : Strategy & Innovation - Maastricht University                         Maastricht, Holland      

•  Coursework in risk- and portfolio management, M&A, strategic business expansion.
•
•  GPA: 8.55/10.

Thesis: The impact of leadership on talent retention strategies.

•  Coursework in statistical modelling, thought leadership.
•  GPA: 7.5/10.

Sep 2018 - Jun 2021                       B.Sc : International Business - Maastricht University                         Maastricht, Holland     

•  Coursework in capital and VC funding, due diligence.
•  Erasmus semester at SGH
•  GPA: 8.2/10.



                                 Warsaw, Poland

Languages

English: full proficiency, German, French, and Spanish: Upper Intermediate, Dutch: Native (EF language exchange in Barcelona,
Madrid, Cambridge, San Diego, Nice)

Core Skills
Microsoft Office Suite, people and communication skills, leadership and can-do attitude, time management

Outdoors sports and fondue, TED talks, organizing own flee market, maintaining several corporate social media accounts

Interests & Other achievements

Volunteering activities



Member of PREMIUM excellence program, the Social Investment Club, and the Beta Gamma Sigma society for honoured students
'''

annotations = skill_extractor.annotate(job_description)
skill_extractor.describe(annotations)

loading full_matcher ...
loading abv_matcher ...
loading full_uni_matcher ...
loading low_form_matcher ...
loading token_matcher ...


c:\Users\darry\anaconda3\envs\FYP\lib\site-packages\skillNer\utils.py:99: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  vec_similarity = token1.similarity(token2)
